In [5]:
import warnings 
warnings.simplefilter('ignore')

import celltypist
import scparadise
import scanpy as sc
import muon as mu
import anndata
import pandas as pd
import numpy as np
import os

In [2]:
os.chdir('/mnt/c/Users/vadim/scRNA/scParadise/scripts_article')

In [3]:
# Load data
adata = sc.read_h5ad('Pancreas/adata.h5ad') 

In [13]:
# Create training and testing datasets
adata_train = adata[adata.obs['donor_id'].isin(['SAMN16365027', 'SAMN17528599'])].copy()
adata_test_1 = adata[adata.obs['donor_id'].isin(['HP-20152-01'])].copy()
adata_test_2 = adata[adata.obs['donor_id'].isin(['HP17225-01T2D'])].copy()
adata_test_3 = adata[adata.obs['donor_id'].isin(['HP19047-01'])].copy()
adata_test_4 = adata[adata.obs['donor_id'].isin(['SAMN11867362'])].copy()
adata_test_5 = adata[adata.obs['donor_id'].isin(['SAMN12022246'])].copy()
adata_test_6 = adata[adata.obs['donor_id'].isin(['SAMN13108021', 'SAMN11642375'])].copy()
adata_test_7 = adata[adata.obs['donor_id'].isin(['SAMN11476721', 'SAMN10873960'])].copy()
adata_test_8 = adata[adata.obs['donor_id'].isin(['SAMN10439569', 'SAMN10737781'])].copy()
adata_test_9 = adata[adata.obs['donor_id'].isin(['SAMN13319813', 'SAMN11522709'])].copy()
adata_test_10 = adata[adata.obs['donor_id'].isin(['HP18304', 'SAMN11157311'])].copy()

# Normalize data, find highly variable features
for i in [adata_train, adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8, adata_test_9, adata_test_10]:
    i.layers['counts'] = i.X.copy()
    sc.pp.normalize_total(i, target_sum=10000)
    sc.pp.log1p(i)
    i.raw = i

In [14]:
# Subset anndata object based on a selected marker genes
genes = pd.read_csv('Pancreas/genes_for_AI.csv')
adata_train = adata_train[:, genes.genes].copy()

In [15]:
# Train and save models for celltype_l1, celltype_l2 and celltype_l3 annotation levels
lst_annotations = ['celltype_l1']
for i in lst_annotations:
    # Training step
    model = celltypist.train(adata_train, labels = i, check_expression=False, feature_selection = False, n_jobs=12)
    # Save the model
    model.write(os.path.join('Pancreas/celltypist_test', i + '.pkl'))

🍳 Preparing data before training
🔬 Input data has 11185 cells and 530 genes
⚖️ Scaling input data
🏋️ Training data using logistic regression
✅ Model training done!


In [16]:
# Create lists with test samples
lst_test_folders = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
lst_test_adatas = [adata_test_1, adata_test_2, adata_test_3, adata_test_4, adata_test_5, adata_test_6, adata_test_7, adata_test_8, adata_test_9, adata_test_10]

In [17]:
for adata_test, folder in zip(lst_test_adatas, lst_test_folders):
    os.makedirs(os.path.join('Pancreas/celltypist_test/reports', folder))
    # Prediction step using pretrained models
    for i in lst_annotations:
        predictions = celltypist.annotate(adata_test, model = os.path.join('Pancreas/celltypist_test', i + '.pkl'), majority_voting = True)
        adata_test = predictions.to_adata()
        scparadise.scnoah.report_classif_full(adata_test, 
                                              celltype = i, 
                                              pred_celltype = 'majority_voting', 
                                              report_name = 'report_celltypist_' + i + '.csv',
                                              save_path = os.path.join('Pancreas/celltypist_test/reports', folder),
                                              save_report = True)

🔬 Input data has 5298 cells and 27484 genes
🔗 Matching reference genes in the model
🧬 530 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 5112 cells and 27484 genes
🔗 Matching reference genes in the model
🧬 530 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering


Successfully saved report



⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 7353 cells and 27484 genes
🔗 Matching reference genes in the model
🧬 530 features used for prediction
⚖️ Scaling input data


Successfully saved report



🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 5549 cells and 27484 genes
🔗 Matching reference genes in the model
🧬 530 features used for prediction
⚖️ Scaling input data
🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering


...

Successfully saved report



🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!
🔬 Input data has 9005 cells and 27484 genes
🔗 Matching reference genes in the model
🧬 530 features used for prediction
⚖️ Scaling input data


Successfully saved report



🖋️ Predicting labels
✅ Prediction done!
👀 Can not detect a neighborhood graph, will construct one before the over-clustering
⛓️ Over-clustering input data with resolution set to 10
🗳️ Majority voting the predictions
✅ Majority voting done!


Successfully saved report



In [11]:
import session_info
session_info.show()